In [1]:
import openai
import json
import os
import time
import csv
from pathlib import Path

#OpenAI API KEy
openai.api_key = "sk-proj-"


In [2]:
def load_prompt(prompt_path):
    with open(prompt_path, 'r', encoding='utf-8') as f:
        return f.read()

def generate_sentences(prompt_text):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt_text}
        ],
        temperature=0.7
    )
    content = response["choices"][0]["message"]["content"]
    try:
        return json.loads(content)
    except json.JSONDecodeError:
        print("JSON parse hatası:", content[:20])
        return []


In [5]:
def generate_schema_sentences(prompt_path, output_json_path, output_csv_path, total_needed=20, batch_size=10):
    base_prompt = load_prompt(prompt_path)
    collected = []

    while len(collected) < total_needed:
        needed = min(batch_size, total_needed - len(collected))
        current_prompt = base_prompt.replace("Generate 30", f"Generate {needed}")
        new_sentences = generate_sentences(current_prompt)
        collected.extend([s for s in new_sentences if isinstance(s, str)])
        collected = list(dict.fromkeys(collected))  # Duplicate'leri temizle
        print(f"{prompt_path.name}: {len(collected)}/{total_needed} cümle toplandı.")
        time.sleep(1)

    # JSON olarak kaydet
    with open(output_json_path, "w", encoding="utf-8") as f:
        json.dump(collected[:total_needed], f, ensure_ascii=False, indent=2)

    # CSV olarak da kaydet
    with open(output_csv_path, "w", encoding="utf-8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Sentence"])
        for s in collected[:total_needed]:
            writer.writerow([s])

    print(f"Tamamlandı: {prompt_path.stem} → {total_needed} cümle kaydedildi.")

In [6]:
from pathlib import Path

prompt_folder = Path("prompts")
output_folder = Path("evaluation")
output_folder.mkdir(exist_ok=True)


for prompt_file in prompt_folder.glob("*.txt"):  
    name = prompt_file.stem 
    json_path = output_folder / f"{name}.json"  
    csv_path = output_folder / f"{name}.csv" 
    generate_schema_sentences(prompt_file, json_path, csv_path) 


AB.txt: 10/20 cümle toplandı.
AB.txt: 20/20 cümle toplandı.
Tamamlandı: AB → 20 cümle kaydedildi.
AS.txt: 10/20 cümle toplandı.
AS.txt: 20/20 cümle toplandı.
Tamamlandı: AS → 20 cümle kaydedildi.
DI.txt: 10/20 cümle toplandı.
DI.txt: 20/20 cümle toplandı.
Tamamlandı: DI → 20 cümle kaydedildi.
DS.txt: 10/20 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 10/20 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 10/20 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 10/20 cümle toplandı.
JSON parse hatası: [
"I constantly feel
DS.txt: 10/20 cümle toplandı.
DS.txt: 20/20 cümle toplandı.
Tamamlandı: DS → 20 cümle kaydedildi.
JSON parse hatası: [
"I feel like no on
ED.txt: 0/20 cümle toplandı.
ED.txt: 10/20 cümle toplandı.
JSON parse hatası: [
"I feel so alone, 
ED.txt: 10/20 cümle toplandı.
ED.txt: 20/20 cümle toplandı.
Tamamlandı: ED → 20 cümle kaydedildi.
EI.txt: 10/20 cümle toplandı.
EI.txt: 20/20 cümle toplandı.
Tamamlandı: EI → 20 cümle kayde

In [7]:
import pandas as pd

output_dir = Path("evaluation")
merged_rows = []

for csv_file in output_dir.glob("*.csv"):
    schema_name = csv_file.stem.upper().replace("_", " ")
    df = pd.read_csv(csv_file)
    df.insert(0, "Stage", schema_name)  # İlk sütuna Stage ekle
    merged_rows.append(df)

merged_df = pd.concat(merged_rows, ignore_index=True)
merged_df.to_csv("evaluation_data.csv", index=False, encoding="utf-8")

print(f"Birleştirme tamamlandı: {len(merged_df)} cümle `evaluation_data.csv` dosyasına kaydedildi.")

Birleştirme tamamlandı: 360 cümle `evaluation_data.csv` dosyasına kaydedildi.


In [ ]:
import pandas as pd

# Boş liste oluştur
data = []

# TXT dosyasını satır satır oku, ilk satırı atla
with open("young_sentence.txt", "r", encoding="utf-8") as file:
    for i, line in enumerate(file):
        if i == 0:
            continue  # İlk satırı atla
        parts = line.strip().split(",", 1)
        if len(parts) == 2:
            stage, sentence = parts
            data.append({"Stage": stage.strip(), "Sentence": sentence.strip()})

# DataFrame oluştur
df = pd.DataFrame(data)

# CSV'ye kaydet
df.to_csv("stage_sentence.csv", index=False)

In [ ]:
import pandas as pd

# Dosyaları oku
df1 = pd.read_csv("evaluation_data.csv")   # İlk CSV
df2 = pd.read_csv("stage_sentence.csv")    # İkinci CSV

# Gerekirse sütun adlarını kontrol et ve eşleştir
df2 = df2[["Stage", "Sentence"]]           # Sadece gerekli sütunları al

# Satır bazlı birleştirme (alt alta)
merged_df = pd.concat([df1, df2], axis=0, ignore_index=True)

# Sonuç dosyasını kaydet
merged_df.to_csv("evaluation.csv", index=False)